# How to add new diagnostics

When you get more familiar with running this package and think to create your own diagnostics, just follow this guide to build new diagnostics based on set5 diags.

1\. Specify pressure levels[units: mb] for 3-d variables: 
For example, in mydiags.json, set:
```
"levels": [200.0,850.0]
```
You can put any level desired for diagnostics.

or,

set pressure level from command line:
```
set5_driver.py -p myparams.py -d mydiags.json -levels 200.0
```

2\. Specify regions for diagnostics:

For example, in mydiags.json, set:
```
"region": ['TROPICS','ocean_TROPICS']
```

or,

set pressure level from command line:
```
set5_driver.py -p myparams.py -d mydiags.json -region 'TROPICS'
```
A list of built-in default regions can be found here [link]

3\. Add User derived variables [More advanced]



```python

```

```json

```